## Zero Bug Fix
* Details on issue can be found [here](https://github.com/defipy-devs/uniswappy/issues/50)

In [1]:
from uniswappy import *
import pandas as pd
import numpy as np
from pydantic import BaseModel, ConfigDict
from dataclasses import dataclass
import traceback
import matplotlib.pyplot as plt

In [2]:
@dataclass
class TokenScenario:
    user: str
    usdt_in: int
    reserve0: int
    reserve1: int
    name0: str
    name1: str
    address0: str
    address1: str
    tick_spacing: any
    fee: any
    init_price: any

def plotme(df, title, annotation):
    plt.figure(figsize=(10,6))
    for key, grp in df.groupby('lower'):
        plt.plot(grp['swap'], grp['price'], label=key)

    plt.text(25000, 0.6, annotation )
    plt.legend(title=title)
    plt.xlabel('Tokens sold (TKN)')
    plt.ylabel('Swap price (USDT/TKN)')
    plt.show()

def get_tick(lp, x):
    if x == "min_tick":
        return UniV3Utils.getMinTick(lp.tickSpacing)
    elif x == "max_tick":
        return UniV3Utils.getMaxTick(lp.tickSpacing)
    return UniV3Helper().get_price_tick(lp, 0, x)

def setup_lp(tenv, pool_params):
    factory = UniswapFactory("ETH pool factory", "0x%d" )
    tkn0 = ERC20(tenv.name0, tenv.address0)
    tkn1 = ERC20(tenv.name1, tenv.address1)
    exchg_data = UniswapExchangeData(
        tkn0 = tkn0, tkn1 = tkn1,
        symbol="LP", 
        address="0x011", version="V3",
        tick_spacing = tenv.tick_spacing,
        fee=tenv.fee
    )
    lp = factory.deploy(exchg_data)
    #lp.initialize(tenv.init_price)
    for pool_param in pool_params:
        
        # AddLiquidity().apply(
        #     lp, tkn1, tenv.user, pool_param[0],
        #     get_tick(lp, pool_param[1]), 
        #     get_tick(lp, pool_param[2])
        # )
        # lp.summary()

        init_p = tenv.reserve1/tenv.reserve0
        lwr_tick = UniV3Helper().get_price_tick(lp, -1, init_p, 1000)
        upr_tick = UniV3Helper().get_price_tick(lp, 1, init_p, 1000)
        
        Join().apply(
            lp, tenv.user, tenv.reserve0, tenv.reserve1, 
            lwr_tick, upr_tick)

    return (lp, tkn0, tkn1)

def do_calc(tenv):
    results = []
    for lower in [0.95, 0.9, 0.8, 0.7, 0.5, 0.1, 0.000001]:
        for swap in np.geomspace(100, tenv.usdt_in, num=100):
            (lp, tkn0, tkn1) = setup_lp(tenv, [[
                tenv.reserve, lower, 1.0
            ]])
            try:
                out = Swap().apply(lp, tkn0, tenv.user, swap)
                results.append({
                    "lower": lower,
                    "swap": swap,
                    "out": out,
                    "price": float(out) / float(swap)
                })
            except AssertionError:
                pass
    return pd.DataFrame(results)

def do_calc1(tenv):
    results = []
    insurance_lower = 0.95
    insurance_upper = 0.98
    for frac_reserve in np.geomspace(0.0001, 0.99, num=10):
        for swap in np.geomspace(100, tenv.usdt_in, num=100):
            (lp, tkn0, tkn1) = setup_lp(
                tenv, [
                    [tenv.reserve * (1.0 - frac_reserve), "min_tick", 1.0],
                    [tenv.reserve * frac_reserve, insurance_lower, 1.0]
                ]
            )
            try:
                out = Swap().apply(lp, tkn0, tenv.user, swap)
                results.append({
                    "lower": frac_reserve * 100,
                    "swap": swap,
                    "out": out,
                    "price": float(out) / float(swap)
                })
            except AssertionError:
                pass
    return pd.DataFrame(results)

__all__ = ['plotme', 'do_calc', 'do_calc1', 'setup_lp', 'TokenScenario']

In [3]:
fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
init_price = UniV3Utils.encodePriceSqrt(1000,1000)

usdt_in = 10**6
tenv = TokenScenario(
    user = 'user',
    reserve0 = usdt_in * 0.2,
    reserve1 = usdt_in * 0.2,
    name0 = "TKN",
    name1 = "USDT",
    address0 = "0x111",
    address1 = "0x09",   
    usdt_in = usdt_in,
    tick_spacing = tick_spacing,
    fee = fee,
    init_price = init_price
)

nsteps = 50
lp_prices = []
lp_liquidity = []
lp_swap = []

def do_sim(tenv, lp, tkn0, tkn1):
    # Set up liquidity pool
    frac_reserve = 0.05

    # Run simulation
    for i in range(1, nsteps):
        accounts = MockAddress().apply(500)
        select_tkn = EventSelectionModel().bi_select(0.5)
        rnd_add_amt = TokenDeltaModel(10000).delta()
        rnd_swap_amt = TokenDeltaModel(10000).delta()
        user_add = random.choice(accounts)
        user_swap = random.choice(accounts)
        try:
            out = Swap().apply(lp, tkn0 if select_tkn == 0 else tkn1, user_swap, rnd_swap_amt)
            lp_prices.append(lp.get_price(tkn0))
            lp_liquidity.append(lp.get_liquidity())
            lp_swap.append(rnd_swap_amt)
            lp.summary()
            print("*****", select_tkn, rnd_swap_amt, out, lp.get_price(tkn0))
        except AssertionError:
            break
            print("Failed", select_tkn, rnd_swap_amt)
#            print(traceback.format_exc())
            pass

(lp, tkn0, tkn1) = setup_lp(
        tenv, [
            [tenv.reserve0, tenv.reserve1, 0.9, 1.0]
        ]
)
do_sim(tenv, lp, tkn0, tkn1)

Exchange TKN-USDT (LP)
Real Reserves:   TKN = 198759.98127403576, USDT = 201244.13349481538
Gross Liquidity: 4022614.62327768 

***** 1 1244.133494815546 1240.0187259640736 1.0006168089432623
Exchange TKN-USDT (LP)
Real Reserves:   TKN = 196495.40481547554, USDT = 203518.20574881395
Gross Liquidity: 4022614.62327768 

***** 1 2274.0722539985754 2264.5764585602205 1.0017447261376466
Exchange TKN-USDT (LP)
Real Reserves:   TKN = 198173.73963422715, USDT = 201842.68406443472
Gross Liquidity: 4022614.62327768 

***** 0 1678.334818751623 1675.5216843792352 1.0009111221666658
Exchange TKN-USDT (LP)
Real Reserves:   TKN = 198885.580363335, USDT = 201132.4575908677
Gross Liquidity: 4022614.62327768 

***** 0 711.8407291078387 710.2264735670296 1.000557875674148
Exchange TKN-USDT (LP)
Real Reserves:   TKN = 200491.53438881, USDT = 199531.0657782258
Gross Liquidity: 4022614.62327768 

***** 0 1605.9540254749963 1601.3918126418791 0.9997616176010907
Exchange TKN-USDT (LP)
Real Reserves:   TKN = 1